In [1]:
import pandas as pd
from collections import Counter

### Data Input

#### Load all orders, including prior, train, test

In [2]:
orders = pd.read_csv("orders.csv")
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


#### Summary statistics

In [3]:
orders.shape

(3421083, 7)

In [4]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [5]:
orders_prior = orders.copy()
orders_prior=orders_prior[orders_prior.eval_set == 'prior']
orders_prior.drop('eval_set', axis = 1, inplace = True)
orders_prior.head(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


#### load products/items information in individual orders

In [6]:
products_prior = pd.read_csv("order_products__prior.csv")
products_prior.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


#### Count the number of items in each individual order. 

In [7]:
products_user_prior = orders_prior[['order_id', 'user_id']].merge(
    products_prior[['order_id', 'product_id', 'add_to_cart_order']], on = 'order_id')


In [8]:
products_user_length_prior = products_user_prior[['order_id', 'user_id','product_id']].merge(
    products_prior[['order_id', 'add_to_cart_order']].groupby('order_id').max().rename(
        {'add_to_cart_order': 'order_length_prior'}, axis = 1), on = 'order_id')
products_user_length_prior

,order_id,user_id,product_id,order_length_prior
0,2539329,1,196,5
1,2539329,1,14084,5
2,2539329,1,12427,5
3,2539329,1,26088,5
4,2539329,1,26405,5
...,...,...,...,...
32434484,2977660,206209,14197,9
32434485,2977660,206209,38730,9
32434486,2977660,206209,31477,9
32434487,2977660,206209,6567,9


#### Create two variables—$occurrence$ and $total\_occurrence$. 
For an individual customer, $occurrence$ is the number of times an item has been purchased in all his/her prior orders. $total\_occurrence$ is the sum of $occurrence$ for all his/her items.  

In [9]:
count_series = products_user_length_prior.groupby(['user_id', 'product_id']).size()
new_count_series = count_series.to_frame(name = 'occurrence').reset_index()
new_count_series['total_occurrence']=new_count_series.groupby('user_id')['occurrence'].transform('sum')
new_count_series.to_csv('new_count_series.csv',index=False)

### Top 2 picks for each customer
We picked the most two frequently purchased items. Note that $keep='all'$ was used in the $nlargest$ function. 

In [10]:
top_2_picks=new_count_series.set_index(['product_id','total_occurrence']).groupby('user_id').occurrence.nlargest(2,keep='all').reset_index()
top_2_picks=top_2_picks[['user_id','product_id']]
top_2_picks=top_2_picks.applymap(str)
top_2_picks=top_2_picks.groupby(['user_id']).apply(lambda x: ' '.join(x['product_id'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'product_id'})
top_2_picks

,user_id,product_id
0,1,196 12427
1,10,16797 28535 30489 46979 47526
2,100,21616 27344
3,1000,14870 26165 28465 30492 49683
4,10000,21137 5077
...,...,...
206204,99995,9741 1940 20544 26209
206205,99996,23915 49275
206206,99997,1511 3469 20119 27690 28204 40604 41658
206207,99998,31717 38383


In [11]:
top_2_picks_reshaped=top_2_picks.sort_values('user_id')
top_2_picks_reshaped=top_2_picks_reshaped.assign(product_id=top_2_picks_reshaped.product_id.str.split()).explode('product_id')
top_2_picks_reshaped

,user_id,product_id
0,1,196
0,1,12427
1,10,16797
1,10,28535
1,10,30489
...,...,...
206206,99997,41658
206207,99998,31717
206207,99998,38383
206208,99999,30233


#### Directly output the top 2 picks, without attaching any rules to the top 2 picks. 

Identify top 2 picks

In [12]:
top_2_picks_reshaped_no_rules=top_2_picks_reshaped.copy()
top_2_picks_reshaped_no_rules

,user_id,product_id
0,1,196
0,1,12427
1,10,16797
1,10,28535
1,10,30489
...,...,...
206206,99997,41658
206207,99998,31717
206207,99998,38383
206208,99999,30233


In [13]:
top_2_picks_reshaped_no_rules = top_2_picks_reshaped_no_rules.applymap(str)
top_2_picks_reshaped_no_rules = top_2_picks_reshaped_no_rules.groupby(['user_id']).apply(lambda x: ' '.join(x['product_id'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'products'})
top_2_picks_reshaped_no_rules['products']=top_2_picks_reshaped_no_rules['products'].apply(lambda x: ' '.join(set([y.strip() for y in x.split(' ')])))
top_2_picks_reshaped_no_rules


,user_id,products
0,1,12427 196
1,10,28535 47526 46979 16797 30489
2,100,27344 21616
3,1000,14870 26165 30492 49683 28465
4,10000,5077 21137
...,...,...
206204,99995,1940 20544 26209 9741
206205,99996,49275 23915
206206,99997,41658 3469 1511 28204 27690 40604 20119
206207,99998,38383 31717


Merge top 2 picks with orders labelled with $test$ under the variable $eval\_set$

In [14]:
test_orders=orders[orders['eval_set']=='test']
test_orders=test_orders[['order_id','user_id']]
test_orders=test_orders.applymap(str)

In [15]:
test_orders_products_2_picks_no_rules = test_orders.merge(top_2_picks_reshaped_no_rules, how = 'left', left_on='user_id', right_on='user_id')
test_orders_products_2_picks_no_rules['order_id'] = test_orders_products_2_picks_no_rules['order_id'].astype(int)
test_orders_products_2_picks_no_rules.sort_values('order_id')
test_orders_products_2_picks_no_rules=test_orders_products_2_picks_no_rules.applymap(str)
test_orders_products_2_picks_no_rules=test_orders.merge(test_orders_products_2_picks_no_rules,how='left',on='user_id').rename(columns={'order_id_x': 'order_id'})
test_orders_products_2_picks_no_rules


,order_id,user_id,order_id_y,products
0,2774568,3,2774568,39190 47766
1,329954,4,329954,37646 42329 43704 17769 22199 11865 25146 7350...
2,1528013,6,1528013,21903 38293
3,1376945,11,1376945,8309 27959
4,1356845,12,1356845,13176 14992 7076 10863
...,...,...,...,...
74995,2728930,206202,2728930,26620 2693
74996,350108,206204,350108,43961 30561
74997,1043943,206206,1043943,47011 38530
74998,2821651,206207,2821651,44632 13176 3397 36011


In [16]:
test_orders_products_2_picks_no_rules.fillna('None',inplace=True)
test_orders_products_2_picks_no_rules['products']=test_orders_products_2_picks_no_rules['products'].apply(lambda x: x.lstrip())
test_orders_products_2_picks_no_rules['order_id'] = test_orders_products_2_picks_no_rules['order_id'].astype(int)
test_orders_products_2_picks_no_rules=test_orders_products_2_picks_no_rules.sort_values('order_id')
test_orders_products_2_picks_no_rules[['order_id','products']].to_csv('test_orders_products_2_picks_no_rules.csv',index=False)


Attach association rules (min_support = 0.01) to the top 2 picks

In [17]:
rules_01=pd.read_csv("rules/rules_01.csv")
rules_01=rules_01[rules_01['lift']>1]
rules_01=rules_01[['item_A', 'item_B', 'lift']].applymap(str)
rules_01

,item_A,item_B,lift
0,29126,36361,9.449867641226248
1,7076,17766,6.02622930097333
2,12820,11212,5.546732115600253
3,32201,44781,5.3182304913338445
4,28613,45636,5.317848916924218
...,...,...,...
203,38985,22969,1.0210959303830534
204,38544,37718,1.0161856460094716
205,5491,4812,1.0158151019352435
206,38544,4962,1.0125284989322474


In [18]:
top_2_picks_reshaped_01=top_2_picks_reshaped.merge(
    rules_01, how='left', left_on='product_id', right_on='item_A')[['user_id','product_id','item_B']].merge(
    rules_01, how='left', left_on='product_id', right_on='item_B')[['user_id','product_id','item_B_x','item_A']]

top_2_picks_reshaped_01=top_2_picks_reshaped_01.fillna("")
top_2_picks_reshaped_01


,user_id,product_id,item_B_x,item_A
0,1,196,,
1,1,12427,,
2,10,16797,,
3,10,28535,,
4,10,30489,,
...,...,...,...,...
748482,99997,41658,,
748483,99998,31717,,
748484,99998,38383,,
748485,99999,30233,,


In [19]:
top_2_picks_reshaped_01=top_2_picks_reshaped_01.applymap(str)
top_2_picks_reshaped_01["products"] = top_2_picks_reshaped_01[['product_id', 'item_B_x', 'item_A']].agg(' '.join, axis=1)
top_2_picks_reshaped_01 = top_2_picks_reshaped_01[['user_id','products']]
top_2_picks_reshaped_01 = top_2_picks_reshaped_01.groupby(['user_id']).apply(lambda x: ' '.join(x['products'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'products'})
top_2_picks_reshaped_01['products']=top_2_picks_reshaped_01['products'].apply(lambda x: ' '.join(set([y.strip() for y in x.split(' ')])))
top_2_picks_reshaped_01


,user_id,products
0,1,12427 196
1,10,28535 47526 46979 16797 30489
2,100,27344 21616
3,1000,14870 26165 30492 49683 28465
4,10000,21137 5077
...,...,...
206204,99995,26209 1940 20544 9741
206205,99996,49275 23915
206206,99997,41658 3469 1511 28204 27690 40604 20119
206207,99998,38383 31717


Merge (top 2 picks + rules_0.01) with orders labelled with $test$ under the variable $eval\_set$. 

In [20]:
test_orders_products_2_picks_01 = test_orders.merge(top_2_picks_reshaped_01, how = 'left', left_on='user_id', right_on='user_id')
test_orders_products_2_picks_01['order_id'] = test_orders_products_2_picks_01['order_id'].astype(int)
test_orders_products_2_picks_01.sort_values('order_id')
test_orders_products_2_picks_01=test_orders_products_2_picks_01.applymap(str)
test_orders_products_2_picks_01=test_orders.merge(test_orders_products_2_picks_01,how='left',on='user_id').rename(columns={'order_id_x': 'order_id'})
test_orders_products_2_picks_01


,order_id,user_id,order_id_y,products
0,2774568,3,2774568,39190 47766
1,329954,4,329954,7026 37646 42329 43704 17769 22199 11865 2514...
2,1528013,6,1528013,21903 38293
3,1376945,11,1376945,8309 27959
4,1356845,12,1356845,7076 17766 14992 13176 10863
...,...,...,...,...
74995,2728930,206202,2728930,26620 2693
74996,350108,206204,350108,43961 30561
74997,1043943,206206,1043943,47011 38530
74998,2821651,206207,2821651,44632 36011 3397 13176


In [21]:
test_orders_products_2_picks_01.fillna('None',inplace=True)
test_orders_products_2_picks_01['products']=test_orders_products_2_picks_01['products'].apply(lambda x: x.lstrip())
test_orders_products_2_picks_01['order_id'] = test_orders_products_2_picks_01['order_id'].astype(int)
test_orders_products_2_picks_01=test_orders_products_2_picks_01.sort_values('order_id')
test_orders_products_2_picks_01[['order_id','products']].to_csv('test_orders_products_2_picks_01.csv',index=False)


### Top 3 picks for each customer

Identify top 3 picks

In [22]:
top_3_picks=new_count_series.set_index(['product_id','total_occurrence']).groupby('user_id').occurrence.nlargest(3,keep='all').reset_index()
top_3_picks=top_3_picks[['user_id','product_id']]
top_3_picks=top_3_picks.applymap(str)
top_3_picks=top_3_picks.groupby(['user_id']).apply(lambda x: ' '.join(x['product_id'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'product_id'})
top_3_picks

,user_id,product_id
0,1,196 12427 10258
1,10,16797 28535 30489 46979 47526
2,100,21616 27344 24852 29487 30795
3,1000,14870 26165 28465 30492 49683
4,10000,21137 5077 42828
...,...,...
206204,99995,9741 1940 20544 26209
206205,99996,23915 49275 26795
206206,99997,1511 3469 20119 27690 28204 40604 41658
206207,99998,31717 38383 16617 19678


In [23]:
top_3_picks_reshaped=top_3_picks.sort_values('user_id')
top_3_picks_reshaped=top_3_picks_reshaped.assign(product_id=top_3_picks_reshaped.product_id.str.split()).explode('product_id')
top_3_picks_reshaped


,user_id,product_id
0,1,196
0,1,12427
0,1,10258
1,10,16797
1,10,28535
...,...,...
206207,99998,16617
206207,99998,19678
206208,99999,30233
206208,99999,10831


Attach association rules (min_support = 0.01) to the top 3 picks

In [24]:
top_3_picks_reshaped_01=top_3_picks_reshaped.merge(
    rules_01, how='left', left_on='product_id', right_on='item_A')[['user_id','product_id','item_B']].merge(
    rules_01, how='left', left_on='product_id', right_on='item_B')[['user_id','product_id','item_B_x','item_A']]

top_3_picks_reshaped_01=top_3_picks_reshaped_01.fillna("")
top_3_picks_reshaped_01


,user_id,product_id,item_B_x,item_A
0,1,196,,
1,1,12427,,
2,1,10258,,
3,10,16797,,
4,10,28535,,
...,...,...,...,...
1104010,99998,16617,,
1104011,99998,19678,,
1104012,99999,30233,,
1104013,99999,10831,,


In [25]:
top_3_picks_reshaped_01=top_3_picks_reshaped_01.applymap(str)
top_3_picks_reshaped_01["products"] = top_3_picks_reshaped_01[['product_id', 'item_B_x', 'item_A']].agg(' '.join, axis=1)
top_3_picks_reshaped_01 = top_3_picks_reshaped_01[['user_id','products']]
top_3_picks_reshaped_01 = top_3_picks_reshaped_01.groupby(['user_id']).apply(lambda x: ' '.join(x['products'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'products'})
top_3_picks_reshaped_01['products']=top_3_picks_reshaped_01['products'].apply(lambda x: ' '.join(set([y.strip() for y in x.split(' ')])))
top_3_picks_reshaped_01


,user_id,products
0,1,10258 12427 196
1,10,28535 47526 46979 16797 30489
2,100,21616 24852 30795 29487 27344
3,1000,14870 26165 30492 49683 28465
4,10000,21137 42828 5077
...,...,...
206204,99995,26209 1940 20544 9741
206205,99996,26795 49275 23915
206206,99997,41658 3469 1511 28204 27690 40604 20119
206207,99998,16617 38383 19678 31717


Merge (top 3 picks + rules_0.01) with orders labelled with $test$ under the variable $eval\_set$. 

In [26]:
test_orders_products_3_picks_01 = test_orders.merge(top_3_picks_reshaped_01, how = 'left', left_on='user_id', right_on='user_id')
test_orders_products_3_picks_01['order_id'] = test_orders_products_3_picks_01['order_id'].astype(int)
test_orders_products_3_picks_01.sort_values('order_id')
test_orders_products_3_picks_01=test_orders_products_3_picks_01.applymap(str)
test_orders_products_3_picks_01=test_orders.merge(test_orders_products_3_picks_01,how='left',on='user_id').rename(columns={'order_id_x': 'order_id'})
test_orders_products_3_picks_01


,order_id,user_id,order_id_y,products
0,2774568,3,2774568,39190 47766 21903
1,329954,4,329954,7026 37646 42329 43704 17769 22199 11865 2514...
2,1528013,6,1528013,27521 48679 20323 49401 40992 8424 11068 4500...
3,1376945,11,1376945,8309 14947 27959 35948
4,1356845,12,1356845,7076 17766 14992 13176 10863
...,...,...,...,...
74995,2728930,206202,2728930,24852 26620 2693
74996,350108,206204,350108,10199 42828 5646 15649 43961 49075 21137 4828...
74997,1043943,206206,1043943,47011 11520 38530 27086
74998,2821651,206207,2821651,44632 36011 3397 13176


In [27]:
test_orders_products_3_picks_01.fillna('None',inplace=True)
test_orders_products_3_picks_01['products']=test_orders_products_3_picks_01['products'].apply(lambda x: x.lstrip())
test_orders_products_3_picks_01['order_id'] = test_orders_products_3_picks_01['order_id'].astype(int)
test_orders_products_3_picks_01=test_orders_products_3_picks_01.sort_values('order_id')
test_orders_products_3_picks_01[['order_id','products']].to_csv('test_orders_products_3_picks_01.csv',index=False)


Attach association rules (min_support = 0.005) to the top 3 picks

In [28]:
rules_005=pd.read_csv("rules/rules_005.csv")
rules_005=rules_005[rules_005['lift']>1]
rules_005=rules_005[['item_A', 'item_B', 'lift']].applymap(str)
rules_005

,item_A,item_B,lift
0,3858,15692,36.070928301029674
1,9497,8833,15.575942275042445
2,41349,49570,15.258151965808663
3,39739,11224,13.51631869005914
4,19244,26488,10.755235660068333
...,...,...,...
839,6975,26706,1.005704102865293
840,23025,22969,1.0050262401073322
841,44156,23296,1.0040925025202316
842,7533,38241,1.0013476813705426


In [29]:
top_3_picks_reshaped_005=top_3_picks_reshaped.merge(
    rules_005, how='left', left_on='product_id', right_on='item_A')[['user_id','product_id','item_B']].merge(
    rules_005, how='left', left_on='product_id', right_on='item_B')[['user_id','product_id','item_B_x','item_A']]

top_3_picks_reshaped_005=top_3_picks_reshaped_005.fillna("")
top_3_picks_reshaped_005


,user_id,product_id,item_B_x,item_A
0,1,196,,
1,1,12427,,
2,1,10258,,
3,10,16797,,
4,10,28535,,
...,...,...,...,...
1189144,99998,16617,,
1189145,99998,19678,,
1189146,99999,30233,,
1189147,99999,10831,,


In [30]:
top_3_picks_reshaped_005=top_3_picks_reshaped_005.applymap(str)
top_3_picks_reshaped_005["products"] = top_3_picks_reshaped_005[['product_id', 'item_B_x', 'item_A']].agg(' '.join, axis=1)
top_3_picks_reshaped_005 = top_3_picks_reshaped_005[['user_id','products']]
top_3_picks_reshaped_005 = top_3_picks_reshaped_005.groupby(['user_id']).apply(lambda x: ' '.join(x['products'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'products'})
top_3_picks_reshaped_005['products']=top_3_picks_reshaped_005['products'].apply(lambda x: ' '.join(set([y.strip() for y in x.split(' ')])))
top_3_picks_reshaped_005


,user_id,products
0,1,10258 12427 196
1,10,28535 47526 46979 16797 30489
2,100,21616 24852 30795 29487 27344
3,1000,14870 26165 30492 49683 28465
4,10000,21137 42828 5077
...,...,...
206204,99995,26209 1940 20544 9741
206205,99996,26795 49275 23915
206206,99997,41658 3469 1511 28204 27690 40604 20119
206207,99998,16617 38383 19678 31717


Merge (top 3 picks + rules_0.005) with orders labelled with $test$ under the variable $eval\_set$. 

In [31]:
test_orders_products_3_picks_005 = test_orders.merge(top_3_picks_reshaped_005, how = 'left', left_on='user_id', right_on='user_id')
test_orders_products_3_picks_005['order_id'] = test_orders_products_3_picks_005['order_id'].astype(int)
test_orders_products_3_picks_005.sort_values('order_id')
test_orders_products_3_picks_005=test_orders_products_3_picks_005.applymap(str)
test_orders_products_3_picks_005=test_orders.merge(test_orders_products_3_picks_005,how='left',on='user_id').rename(columns={'order_id_x': 'order_id'})
test_orders_products_3_picks_005


,order_id,user_id,order_id_y,products
0,2774568,3,2774568,39190 47766 21903
1,329954,4,329954,7026 37646 42329 43704 17769 22199 11865 2514...
2,1528013,6,1528013,27521 48679 20323 49401 40992 8424 11068 4500...
3,1376945,11,1376945,8309 14947 27959 35948
4,1356845,12,1356845,7076 17766 14992 28134 45346 13176 10863
...,...,...,...,...
74995,2728930,206202,2728930,24852 26620 2693
74996,350108,206204,350108,10199 42828 5646 15649 43961 49075 21137 4828...
74997,1043943,206206,1043943,47011 11520 38530 27086
74998,2821651,206207,2821651,44632 36011 3397 13176


In [32]:
test_orders_products_3_picks_005.fillna('None',inplace=True)
test_orders_products_3_picks_005['products']=test_orders_products_3_picks_005['products'].apply(lambda x: x.lstrip())
test_orders_products_3_picks_005['order_id'] = test_orders_products_3_picks_005['order_id'].astype(int)
test_orders_products_3_picks_005=test_orders_products_3_picks_005.sort_values('order_id')
test_orders_products_3_picks_005[['order_id','products']].to_csv('test_orders_products_3_picks_005.csv',index=False)


### Top 5 picks for each customer

Identify top 5 picks

In [33]:
top_5_picks=new_count_series.set_index(['product_id','total_occurrence']).groupby('user_id').occurrence.nlargest(5,keep='all').reset_index()
top_5_picks=top_5_picks[['user_id','product_id']]
top_5_picks=top_5_picks.applymap(str)
top_5_picks=top_5_picks.groupby(['user_id']).apply(lambda x: ' '.join(x['product_id'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'product_id'})
top_5_picks

,user_id,product_id
0,1,196 12427 10258 25133 13032 46149
1,10,16797 28535 30489 46979 47526
2,100,21616 27344 24852 29487 30795
3,1000,14870 26165 28465 30492 49683
4,10000,21137 5077 42828 42356 24852
...,...,...
206204,99995,9741 1940 20544 26209 429 1408 2120 4605 5713 ...
206205,99996,23915 49275 26795 40454 41396
206206,99997,1511 3469 20119 27690 28204 40604 41658
206207,99998,31717 38383 16617 19678 2651 4421 4799 8277 17...


In [34]:
top_5_picks_reshaped=top_5_picks.sort_values('user_id')
top_5_picks_reshaped=top_5_picks_reshaped.assign(product_id=top_5_picks_reshaped.product_id.str.split()).explode('product_id')
top_5_picks_reshaped

,user_id,product_id
0,1,196
0,1,12427
0,1,10258
0,1,25133
0,1,13032
...,...,...
206208,99999,10831
206208,99999,14221
206208,99999,15872
206208,99999,32016


Attach association rules (min_support = 0.01) to the top 5 picks

In [35]:
top_5_picks_reshaped_01=top_5_picks_reshaped.merge(
    rules_01, how='left', left_on='product_id', right_on='item_A')[['user_id','product_id','item_B']].merge(
    rules_01, how='left', left_on='product_id', right_on='item_B')[['user_id','product_id','item_B_x','item_A']]

top_5_picks_reshaped_01=top_5_picks_reshaped_01.fillna("")
top_5_picks_reshaped_01


,user_id,product_id,item_B_x,item_A
0,1,196,,
1,1,12427,,
2,1,10258,,
3,1,25133,,
4,1,13032,,
...,...,...,...,...
1808172,99999,10831,,
1808173,99999,14221,,
1808174,99999,15872,,
1808175,99999,32016,,


In [36]:
top_5_picks_reshaped_01=top_5_picks_reshaped_01.applymap(str)
top_5_picks_reshaped_01["products"] = top_5_picks_reshaped_01[['product_id', 'item_B_x', 'item_A']].agg(' '.join, axis=1)
top_5_picks_reshaped_01 = top_5_picks_reshaped_01[['user_id','products']]
top_5_picks_reshaped_01 = top_5_picks_reshaped_01.groupby(['user_id']).apply(lambda x: ' '.join(x['products'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'products'})
top_5_picks_reshaped_01['products']=top_5_picks_reshaped_01['products'].apply(lambda x: ' '.join(set([y.strip() for y in x.split(' ')])))
top_5_picks_reshaped_01


,user_id,products
0,1,25133 13032 46149 196 10258 12427
1,10,28535 47526 46979 16797 30489
2,100,21616 24852 30795 29487 27344
3,1000,14870 26165 30492 49683 28465
4,10000,42828 24852 42356 5077 21137
...,...,...
206204,99995,42431 44234 429 2120 14941 37215 40174 16334 ...
206205,99996,40454 23915 41396 49275 26795
206206,99997,41658 3469 1511 28204 27690 40604 20119
206207,99998,47766 17794 16617 37766 8277 22451 35887 2651...


Merge (top 5 picks + rules_0.01) with orders labelled with $test$ under the variable $eval\_set$. 

In [37]:
test_orders_products_5_picks_01 = test_orders.merge(top_5_picks_reshaped_01, how = 'left', left_on='user_id', right_on='user_id')
test_orders_products_5_picks_01['order_id'] = test_orders_products_5_picks_01['order_id'].astype(int)
test_orders_products_5_picks_01.sort_values('order_id')
test_orders_products_5_picks_01=test_orders_products_5_picks_01.applymap(str)
test_orders_products_5_picks_01=test_orders.merge(test_orders_products_5_picks_01,how='left',on='user_id').rename(columns={'order_id_x': 'order_id'})
test_orders_products_5_picks_01


,order_id,user_id,order_id_y,products
0,2774568,3,2774568,17668 9387 39190 47766 21903
1,329954,4,329954,7026 37646 42329 43704 17769 22199 11865 2514...
2,1528013,6,1528013,27521 48679 20323 49401 40992 8424 11068 4500...
3,1376945,11,1376945,35640 8309 14947 27959 34658 35948 8670 28465...
4,1356845,12,1356845,20350 7076 5746 17766 14992 8239 21616 28134 ...
...,...,...,...,...
74995,2728930,206202,2728930,26620 35640 24852 2693 9450 432
74996,350108,206204,350108,10199 42828 5646 15649 43961 49075 21137 4828...
74997,1043943,206206,1043943,47011 11520 40124 38530 27086
74998,2821651,206207,2821651,44632 36011 33787 3397 13176 39180


In [38]:
test_orders_products_5_picks_01.fillna('None',inplace=True)
test_orders_products_5_picks_01['products']=test_orders_products_5_picks_01['products'].apply(lambda x: x.lstrip())
test_orders_products_5_picks_01['order_id'] = test_orders_products_5_picks_01['order_id'].astype(int)
test_orders_products_5_picks_01=test_orders_products_5_picks_01.sort_values('order_id')
test_orders_products_5_picks_01[['order_id','products']].to_csv('test_orders_products_5_picks_01.csv',index=False)


### Top 10 picks for each customer

Identify top 10 picks

In [39]:
top_10_picks=new_count_series.set_index(['product_id','total_occurrence']).groupby('user_id').occurrence.nlargest(10,keep='all').reset_index()
top_10_picks=top_10_picks[['user_id','product_id']]
top_10_picks=top_10_picks.applymap(str)
top_10_picks=top_10_picks.groupby(['user_id']).apply(lambda x: ' '.join(x['product_id'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'product_id'})
top_10_picks

,user_id,product_id
0,1,196 12427 10258 25133 13032 46149 13176 26088 ...
1,10,16797 28535 30489 46979 47526 1529 9339 25931 ...
2,100,21616 27344 24852 29487 30795 6975 13877 19348...
3,1000,14870 26165 28465 30492 49683 19660 26369 3686...
4,10000,21137 5077 42828 42356 24852 42561 49235 28985...
...,...,...
206204,99995,9741 1940 20544 26209 429 1408 2120 4605 5713 ...
206205,99996,23915 49275 26795 40454 41396 36 20679 24852 2...
206206,99997,1511 3469 20119 27690 28204 40604 41658 3896 4...
206207,99998,31717 38383 16617 19678 2651 4421 4799 8277 17...


In [40]:
top_10_picks_reshaped=top_10_picks.sort_values('user_id')
top_10_picks_reshaped=top_10_picks_reshaped.assign(product_id=top_10_picks_reshaped.product_id.str.split()).explode('product_id')
top_10_picks_reshaped

,user_id,product_id
0,1,196
0,1,12427
0,1,10258
0,1,25133
0,1,13032
...,...,...
206208,99999,45007
206208,99999,6046
206208,99999,19048
206208,99999,24852


Attach association rules (min_support = 0.01) to the top 10 picks

In [41]:
top_10_picks_reshaped_01=top_10_picks_reshaped.merge(
    rules_01, how='left', left_on='product_id', right_on='item_A')[['user_id','product_id','item_B']].merge(
    rules_01, how='left', left_on='product_id', right_on='item_B')[['user_id','product_id','item_B_x','item_A']]

top_10_picks_reshaped_01=top_10_picks_reshaped_01.fillna("")
top_10_picks_reshaped_01


,user_id,product_id,item_B_x,item_A
0,1,196,,
1,1,12427,,
2,1,10258,,
3,1,25133,,
4,1,13032,,
...,...,...,...,...
3413413,99999,45007,,
3413414,99999,6046,,
3413415,99999,19048,,
3413416,99999,24852,,


In [42]:
top_10_picks_reshaped_01=top_10_picks_reshaped_01.applymap(str)
top_10_picks_reshaped_01["products"] = top_10_picks_reshaped_01[['product_id', 'item_B_x', 'item_A']].agg(' '.join, axis=1)
top_10_picks_reshaped_01 = top_10_picks_reshaped_01[['user_id','products']]
top_10_picks_reshaped_01 = top_10_picks_reshaped_01.groupby(['user_id']).apply(lambda x: ' '.join(x['products'])).reset_index().rename(columns={'user_id': 'user_id', 0: 'products'})
top_10_picks_reshaped_01['products']=top_10_picks_reshaped_01['products'].apply(lambda x: ' '.join(set([y.strip() for y in x.split(' ')])))
top_10_picks_reshaped_01


,user_id,products
0,1,25133 26405 13032 46149 196 10258 26088 12427...
1,10,1529 28535 25931 47042 47526 40706 46979 1679...
2,100,13877 38689 44278 29487 28553 28733 33279 193...
3,1000,37646 14870 6873 26165 30492 27336 47734 1966...
4,10000,30720 42828 24852 28985 42356 5077 21137 4256...
...,...,...
206204,99995,42431 44234 429 2120 14941 37215 40174 16334 ...
206205,99996,34202 31608 40454 38996 1408 49275 40348 4377...
206206,99997,18385 5240 17795 9364 41787 8424 41658 27690 ...
206207,99998,47766 17794 16617 37766 8277 22451 35887 2651...


Merge (top 10 picks + rules_0.01) with orders labelled with $test$ under the variable $eval\_set$. 

In [43]:
test_orders_products_10_picks_01 = test_orders.merge(top_10_picks_reshaped_01, how = 'left', left_on='user_id', right_on='user_id')
test_orders_products_10_picks_01['order_id'] = test_orders_products_10_picks_01['order_id'].astype(int)
test_orders_products_10_picks_01.sort_values('order_id')
test_orders_products_10_picks_01=test_orders_products_10_picks_01.applymap(str)
test_orders_products_10_picks_01=test_orders.merge(test_orders_products_10_picks_01,how='left',on='user_id').rename(columns={'order_id_x': 'order_id'})
test_orders_products_10_picks_01


,order_id,user_id,order_id_y,products
0,2774568,3,2774568,32402 18599 17668 9387 16797 22035 39190 2481...
1,329954,4,329954,7026 37646 42329 43704 17769 22199 11865 2514...
2,1528013,6,1528013,27521 48679 20323 49401 40992 8424 11068 4500...
3,1376945,11,1376945,44632 35640 8309 14947 27959 16869 10644 3357...
4,1356845,12,1356845,12797 20350 35345 42450 48835 7120 28134 1220...
...,...,...,...,...
74995,2728930,206202,2728930,26620 35640 38837 24852 12919 17038 2693 4968...
74996,350108,206204,350108,1402 18523 43504 30720 20995 30328 23375 9366...
74997,1043943,206206,1043943,23737 47011 27104 36724 21653 41387 35140 130...
74998,2821651,206207,2821651,44632 32869 27966 36011 2954 39619 11068 3378...


In [44]:
test_orders_products_10_picks_01.fillna('None',inplace=True)
test_orders_products_10_picks_01['products']=test_orders_products_10_picks_01['products'].apply(lambda x: x.lstrip())
test_orders_products_10_picks_01['order_id'] = test_orders_products_10_picks_01['order_id'].astype(int)
test_orders_products_10_picks_01=test_orders_products_10_picks_01.sort_values('order_id')
test_orders_products_10_picks_01[['order_id','products']].to_csv('test_orders_products_10_picks_01.csv',index=False)
